# Overview

This notebook includes prototypical code for the operator overview write-up. It does not depend on any exisiting code in DiffEqOperators, but it should be easy to modify the current codebase to achieve the same functionality.

To make things simpler, everything is assumed to use `Float64` datatype. I will also always use the out-of-place convention (i.e. `*` instead of `A_mul_B!`).

The naming of different operators will use the following convention, as in the writeup:

- `L` denotes a (quasi)linear operator.

- `A` denotes an operator that can generally be affine.

- `b` denotes the bias of an affine operator

- (Not discussed but I guess we may as well make the call?) `M` for raw matrices (`AbstractMatrix`) and `x`, `y`, `u`, etc for vectors.

In [1]:
import Base: +, *

abstract type DiffEqOperator end
abstract type DiffEqLinearOperator <: DiffEqOperator end

# 1. Constant case (i.e. no `update_coefficients!`)

## 1.1 Abstract operator interface

Below is a simplified operator interface used in my new interface draft. Basically we want to express lazy addition and multiplication of linear operators naturally using `+` and `*`. The `as_array` interface returns the most suitable (dense/sparse) representation of the underlying operator as an `AbstractMatrix` (or should we treat this as a type conversion and just use `AbstractMatrix`?).

The affine operator is defined in such a way that arithmetic on them and linear operators always yield an `DiffEqAffineOperator`.

In [2]:
# We should improve type stability by including more inferrable fields in the type signature
# But for the sake of this notebook I'll make things simple
struct DiffEqArrayOperator <: DiffEqLinearOperator
    M::AbstractMatrix{Float64}
end

*(L::DiffEqArrayOperator, x::Vector{Float64}) = L.M * x
as_array(L::DiffEqArrayOperator) = L.M

struct DiffEqOperatorCombination <: DiffEqLinearOperator
    Ls::Tuple{Vararg{DiffEqLinearOperator}}
end

*(L::DiffEqOperatorCombination, x::Vector{Float64}) = sum(Lk -> Lk*x, L.Ls)
as_array(L::DiffEqOperatorCombination) = sum(as_array, L.Ls)

struct DiffEqOperatorComposition <: DiffEqLinearOperator
    Ls::Tuple{Vararg{DiffEqLinearOperator}}
end

*(L::DiffEqOperatorComposition, x::Vector{Float64}) = foldl((u, Lk) -> Lk*u, x, L.Ls)
as_array(L::DiffEqOperatorComposition) = prod(as_array, reverse(L.Ls))

# Arithmetic
# L1 + L2
+(L1::DiffEqOperatorCombination, L2::DiffEqOperatorCombination) = DiffEqOperatorCombination((L1.Ls..., L2.Ls...))
+(L1::DiffEqOperatorCombination, L2::DiffEqLinearOperator) = DiffEqOperatorCombination((L1.Ls..., L2))
+(L1::DiffEqLinearOperator, L2::DiffEqOperatorCombination) = DiffEqOperatorCombination((L1, L2.Ls...))
+(L1::DiffEqLinearOperator, L2::DiffEqLinearOperator) = DiffEqOperatorCombination((L1, L2))

# We can implement '-' after adding in coefficients for the combinations

# L1 * L2
# Note the application order
*(L1::DiffEqOperatorComposition, L2::DiffEqOperatorComposition) = DiffEqOperatorComposition((L2.Ls..., L1.Ls...))
*(L1::DiffEqLinearOperator, L2::DiffEqOperatorComposition) = DiffEqOperatorComposition((L2.Ls..., L1))
*(L1::DiffEqOperatorComposition, L2::DiffEqLinearOperator) = DiffEqOperatorComposition((L2, L1.Ls...))
*(L1::DiffEqLinearOperator, L2::DiffEqLinearOperator) = DiffEqOperatorComposition((L2, L1));

In [3]:
# Affine Operator
struct DiffEqAffineOperator <: DiffEqOperator
    L::DiffEqLinearOperator
    b::Vector{Float64}
end

*(A::DiffEqAffineOperator, x::Vector{Float64}) = A.L * x + A.b

# Arithmetic on affine operators
+(L1::DiffEqLinearOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(L1 + A2.L, A2.b)
+(A1::DiffEqAffineOperator, L2::DiffEqLinearOperator) = DiffEqAffineOperator(A1.L + L2, A1.b)
+(A1::DiffEqAffineOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(A1.L + A2.L, A1.b + A2.b)
*(L1::DiffEqLinearOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(L1 * A2.L, L1 * A2.b)
*(A1::DiffEqAffineOperator, L2::DiffEqLinearOperator) = DiffEqAffineOperator(A1.L * L2, A1.b)
*(A1::DiffEqAffineOperator, A2::DiffEqAffineOperator) = DiffEqAffineOperator(A1.L * A2.L, A1.L * A2.b + A1.b);

## 1.2 Discretization of the differential operator (stencil convolution)

The 2nd-order central difference approximation to $\partial_x^2$ and the 1st-order upwind approximation to $\partial_x$ are included. The general case can be modified from the stencil convolution code in DiffEqOperators.

The upwind operator is implemented differently from DiffEqOperators, where the direction at each point is stored. Here only the left/right operator is constructed and we interpret

$$ \mu(x)\partial_x = \mu^+(x)\partial_x^+ + \mu^-(x)\partial_x^- $$

(Might not be a favorable approach, especially if we wish to extend to multidimensional upwind operators)

In [4]:
struct DiffusionOperator <: DiffEqLinearOperator
    dx::Float64
    m::Int # number of interior points
end

*(L::DiffusionOperator, x::Vector{Float64}) = [x[i] + x[i+2] - 2*x[i+1] for i in 1:L.m] / L.dx^2
as_array(L::DiffusionOperator) = spdiagm((ones(L.m), -2*ones(L.m), ones(L.m)), (0,1,2)) / L.dx^2;

In [5]:
struct DriftOperator <: DiffEqLinearOperator
    dx::Float64
    m::Int # number of interior points
    direction::Bool # true = right, false = left
end

function *(L::DriftOperator, x::Vector{Float64})
    if L.direction # right drift
        [x[i+1] - x[i] for i in 1:L.m] / L.dx
    else # left drift
        [x[i+2] - x[i+1] for i in 1:L.m] / L.dx
    end
end
function as_array(L::DriftOperator)
    if L.direction # right drift
        spdiagm((-ones(L.m), ones(L.m), zeros(L.m)), (0,1,2)) / L.dx
    else # left drift
        spdiagm((zeros(L.m), -ones(L.m), ones(L.m)), (0,1,2)) / L.dx
    end
end;

# 1.3 Boundary extrapolation operator $Q$

Question: is it OK to shorthand "boundary extrapolation operator" as "BEOperator" or simply "BE"?

A generic $Q$ from the generic boundary condition $Bu = b$ can be a bit difficult to implement, but the simple case of Dirichlet/Neumann BC is easy to handle.

It should be easy to modify `AbsorbingBoundaryMap` and `ReflectingBoundaryMap` to incorporate boundaries that are absorbing at one end and reflecting at another.

The non-zero Dirichlet/Neumann Qs do not have their own type. Instead we construct the operator as an affine map, with `AbsorbingBoundaryMap` and `ReflectingBoundaryMap` its linear part.

In [6]:
struct AbsorbingBE <: DiffEqLinearOperator
    m::Int # number of interior points
end

*(Q::AbsorbingBE, x::Vector{Float64}) = [0.0; x; 0.0]
as_array(Q::AbsorbingBE) = sparse([zeros(Q.m)'; eye(Q.m); zeros(Q.m)'])

struct ReflectingBE <: DiffEqLinearOperator
    m::Int # number of interior points
end

*(Q::ReflectingBE, x::Vector{Float64}) = [x[1]; x; x[end]]
as_array(Q::ReflectingBE) = sparse([1.0 zeros(Q.m-1)'; eye(Q.m); zeros(Q.m-1)' 1.0]);

In [7]:
function Dirichlet_BE(m::Int, bl::Float64, br::Float64)
    # y[1] = bl, y[end] = br
    L = AbsorbingBE(m)
    b = [bl; zeros(m); br]
    DiffEqAffineOperator(L, b)
end

function Neumann_BE(m::Int, dx::Float64, bl::Float64, br::Float64)
    # (y[2] - y[1])/dx = bl, (y[end] - y[end-1])/dx = br
    L = ReflectingBE(m)
    b = [-bl*dx; zeros(m); br*dx]
    DiffEqAffineOperator(L, b)
end;

Examples of non-zero BC:

In [8]:
dx = 1.0
u = [1.,2.,3.,4.]
QD = Dirichlet_BE(4, 10.0, 20.0)
QN = Neumann_BE(4, dx, 1.0, 2.0);

In [9]:
QD * u

6-element Array{Float64,1}:
 10.0
  1.0
  2.0
  3.0
  4.0
 20.0

In [10]:
QN * u

6-element Array{Float64,1}:
 0.0
 1.0
 2.0
 3.0
 4.0
 6.0

## 1.4 Constructing operators for the Fokker-Planck equations

The most general case is in section 3.5 of the write-up:

$$ \mathcal{L} = \mu(x)\partial_x + \frac{\sigma(x)^2}{2}\partial_{xx} $$

where the drift term is discretized using upwind operators as described in section 2:

$$ \mu(x)\partial_x = \mu^+(x)\partial_x^+ + \mu^-(x)\partial_x^- $$

The discretized operators are expressed as the composition of an interior stencil convolution operator `L` and boundary extrapolation operator `Q` (they are generally affine). The drift and diffusion coefficients can be expressed as diagonal matrices (wrapped in a `DiffEqArrayOperator`).

The script below can be modified to represent each of the scenarios described in secition 3 of the write-up (except the mixed-BC case).

In [11]:
# The grid
N = 10
dx = 1.0
xs = collect(1:N) * dx # interior nodes

# Discretization of the differential operators
L1p = DriftOperator(dx, N, true)
L1m = DriftOperator(dx, N, false)
L2 = DiffusionOperator(dx, N)

# Boundary operators
Q = AbsorbingBE(N)
# Q = Neumann_BE(N, dx, 1.0, 2.0)

# Coefficients
mu = rand(N)
mup = [mu[i] > 0.0 ? mu[i] : 0.0 for i in 1:N]
mum = [mu[i] < 0.0 ? mu[i] : 0.0 for i in 1:N]
sigma = rand(N) + 1.0

# Construct the final product
Ldrift = DiffEqArrayOperator(Diagonal(mup)) * L1p + DiffEqArrayOperator(Diagonal(mum)) * L1m
Ldiffusion = DiffEqArrayOperator(Diagonal(sigma.^2 / 2)) * L2
A = (Ldrift + Ldiffusion) * Q

DiffEqOperatorComposition((AbsorbingBE(10), DiffEqOperatorCombination((DiffEqOperatorComposition((DriftOperator(1.0, 10, true), DiffEqArrayOperator([0.282496 0.0 … 0.0 0.0; 0.0 0.840896 … 0.0 0.0; … ; 0.0 0.0 … 0.518511 0.0; 0.0 0.0 … 0.0 0.896779]))), DiffEqOperatorComposition((DriftOperator(1.0, 10, false), DiffEqArrayOperator([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]))), DiffEqOperatorComposition((DiffusionOperator(1.0, 10), DiffEqArrayOperator([1.76829 0.0 … 0.0 0.0; 0.0 1.59721 … 0.0 0.0; … ; 0.0 0.0 … 0.676772 0.0; 0.0 0.0 … 0.0 1.25609])))))))

(The standard printout for the composed operator is a bit messy. Should probably implement `Base.show` for the composition types)

In [12]:
# Solve the HJBE (rI - A)u = x
# In the complete version, we can just do (r*I - A) \ u
# For now we will break the access barrier and get dirty
r = 0.05
if isa(A, DiffEqAffineOperator)
    LHS = r*speye(N) - as_array(A.L)
    RHS = xs + A.b
else
    LHS = r*speye(N) - as_array(A)
    RHS = xs
end
u = LHS \ RHS

10-element Array{Float64,1}:
 19.0086
 34.9524
 42.3442
 45.8283
 46.1967
 44.579 
 38.5766
 32.2452
 24.5824
 11.3082